<img src="../../../Other/img/SWA.jpg" width="790" height="410">

标准的衰减学习率策略在前75%的训练时间里进行学习,然后再把学习率设定到一个合理的较大的常数学习率上,在剩余的25%的训练时间里学习.SWA的平均在最后的25%的训练时间里面产生.

***

<img src="../../../Other/img/SWA_0.jpg" width="790" height="210">

Illustrations of SWA and SGD with a Preactivation ResNet-164 on CIFAR-1001. 
Left: test error surface for three FGE samples and the corresponding SWA solution (averaging in weight space).
Middle and Right: test error and train loss surfaces showing the weights proposed by SGD (at convergence) and SWA, starting from the same initialization of SGD after 125 training epochs.


* We show that simple averaging of multiple points along the trajectory of SGD, with a cyclical or constant learning rate, leads to better generalization than conventional training. 

* SWA is extremely easy to implement and has virtually no computational overhead compared to the
conventional training schemes.


### 算法实现

***

**Algorithm** Stochastic Weight Averaging

***

**Require:**

&emsp;&emsp;weights $\hat{w}$,LR bounds $\alpha_1, \alpha_2$

&emsp;&emsp;cycle length $c$ (for constant learning rate $c=1$),number of interation n

**Ensure:** $ w_{SMA} $

&emsp; $ w \leftarrow \hat{w}$ {Initialize weights with $\hat{w}$}

&emsp;$ w_{SWA} \leftarrow w$

&emsp;**for** $ i \leftarrow 1,2, \ldots, n $ **do**

&emsp;&emsp;$\alpha \leftarrow \alpha(i)$ {Calculate LR for the iteration}

&emsp;&emsp;$w \leftarrow w-\alpha \nabla \mathcal{L}_{i}(w)${Stochastic gradient update}

&emsp;&emsp;**if** mod($i$, $c=0$) **then**

&emsp;&emsp;&emsp;$n_{models} \leftarrow i/c${Number of models}

&emsp;&emsp;&emsp;$w_{\text {SWA }} \leftarrow \frac{w_{\text {SWA }} \cdot n_{\text {models }}+w}{n_{\text {models }}+1}$ {Update average}

&emsp;&emsp;**end if**

{Compute BatchNorm statistics for $w_{SWA}$ weights}

In [1]:
import torch
import torch.utils.data as Data
from datasets import load_dataset
from transformers import BertTokenizer
from transformers import BertModel
import torch.optim as optim
import numpy as np
import random

In [2]:
def set_seed(seed):
    """PyTorch随机数种子设置大全"""
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)  # CPU上设置随机种子
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)  # 当前GPU上设置随机种子
        # torch.cuda.manual_seed_all(seed) # 所有GPU上设置随机种子


set_seed(42)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
class Dataset_obj(Data.Dataset):
    """定义数据集"""

    def __init__(self, split):
        self.dataset = load_dataset('seamew/ChnSentiCorp', split=split)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        """定义索引方式"""
        text = self.dataset[i]['text']
        label = self.dataset[i]['label']
        return text, label


dataset = Dataset_obj('train')  # torch.utils.data.Dataset

for text, label in dataset:
    # 调用__getitem__方法
    print(text)
    print(label)
    break

选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般
1


In [5]:
model_ckpt = "bert-base-chinese"

token = BertTokenizer.from_pretrained(model_ckpt)
print(token)

pretrained = BertModel.from_pretrained(model_ckpt)
print(pretrained)

# 冻结网络层参数(不进行梯度更新)
for param in pretrained.parameters():
    param.requires_grad = False

PreTrainedTokenizer(name_or_path='bert-base-chinese', vocab_size=21128, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(21128, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [6]:
def collate_fn(data):
    sents = [i[0] for i in data]
    labels = [i[1] for i in data]

    # 批量编码句子
    data = token(text=sents,
                 truncation=True,
                 padding='max_length',
                 max_length=512,
                 return_token_type_ids=True,
                 return_attention_mask=True,
                 return_tensors='pt')

    input_ids = data['input_ids']
    attention_mask = data['attention_mask']
    token_type_ids = data['token_type_ids']
    labels = torch.LongTensor(labels)
    return input_ids, attention_mask, token_type_ids, labels


# 数据处理
loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=64,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True)

print(len(loader))

for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader):
    print(input_ids)
    print(input_ids.shape)
    print(attention_mask)
    print(token_type_ids)
    print(labels)
    model_result = pretrained(input_ids=input_ids,
                              attention_mask=attention_mask,
                              token_type_ids=token_type_ids)
    print(model_result.last_hidden_state.shape)
    break

150
tensor([[ 101, 1599, 3614,  ...,    0,    0,    0],
        [ 101, 2218, 3221,  ...,    0,    0,    0],
        [ 101, 3193, 7623,  ...,    0,    0,    0],
        ...,
        [ 101,  791, 1921,  ...,    0,    0,    0],
        [ 101, 1372, 3300,  ...,    0,    0,    0],
        [ 101, 6821,  702,  ...,    0,    0,    0]])
torch.Size([64, 512])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
tensor([1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1,
        1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0,
        0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0,

In [7]:
class Model(torch.nn.Module):
    """下游训练任务模型"""

    def __init__(self, pretrained_model):
        super().__init__()
        self.fc = torch.nn.Linear(768, 2)  # 二分类任务
        self.pretrained = pretrained_model

    def forward(self, input_ids, attention_mask, token_type_ids):
        out = self.pretrained(input_ids=input_ids,
                              attention_mask=attention_mask,
                              token_type_ids=token_type_ids)

        out = self.fc(out.pooler_output)
        out = out.softmax(dim=1)  # 模型预测值
        return out

In [8]:
# 损失函数
criterion = torch.nn.CrossEntropyLoss()

model = Model(pretrained)
model = model.to(device)  # 模型设备切换

optimizer = optim.AdamW(model.parameters(), lr=5e-4)  # 优化器
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)  # 余弦退火


###########################################################################
# step 1:
def avg_fn(averaged_model_parameter, model_parameter, num_averaged):
    # averaged_model_parameter,即w_{SWA}_i
    # model_paramter,即w_i
    # num_averaged,即n_{models}
    #
    # 论文算法w_{SWA}更新方式:
    # 解析:
    # swa_start=4;epoch=0,1,2,3,4,5,6,7,8,9
    #
    # epoch=4时;w_{SWA}_1=w_4
    #
    # epoch=5时;n_{models}=1,w_{SWA}_2=(w_{SWA}_1*n_{models}+w_5)/(n_{models+1})
    # 即:w_{SWA}_2 = (w_4 + w_5) / 2
    #
    # epoch=6时;n_{models}=2,w_{SWA}_3=(w_{SWA}_2*n_{models}+w_6)/(n_{models+1})
    # 即:w_{SWA}_3 = (w_4 + w_5 + w_6) / 3
    #
    # epoch=7时;n_{models}=3,w_{SWA}_4=(w_{SWA}_3*n_{models}+w_7)/(n_{models+1})
    # 即:w_{SWA}_4 = (w_4 + w_5 + w_6 + w_7) / 4
    #
    # 以此类推
    return averaged_model_parameter + (model_parameter - averaged_model_parameter) / (num_averaged + 1)


# Implements averaged model for Stochastic Weight Averaging (SWA).
swa_model = torch.optim.swa_utils.AveragedModel(model,
                                                avg_fn=avg_fn)  # the averaging function used to update parameters(可自定义)
# SWA开始时的epoch(当epoch为该值时开始记录模型权重)
swa_start = 4
# Anneals the learning rate in each parameter group to a fixed value.
# SWA开始时使用的学习率策略
'''
Typically, in SWA the learning rate is set to a high constant value. 
SWALR is a learning rate scheduler that anneals the learning rate to a fixed value, and then keeps it constant. 
For example, the following code creates a scheduler that linearly anneals the learning rate from its initial value to 0.05 in 2 epochs within each parameter group:
'''
swa_scheduler = torch.optim.swa_utils.SWALR(optimizer=optimizer,
                                            # swa_lrs (float or list): the learning rate value for all param groupstogether or separately for each group.
                                            swa_lr=0.05,
                                            # anneal_epochs (int): number of epochs in the annealing phase (default: 10)
                                            anneal_epochs=2,
                                            # anneal_strategy (str): "cos" or "linear"; specifies the annealing strategy: "cos" for cosine annealing, "linear" for linear annealing(default: "cos")
                                            anneal_strategy="linear")
###########################################################################

# 模型训练
model.train()

for epoch in range(10):
    print("**************************epoch:{}**************************".format(epoch))
    for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader):
        # 数据设备切换
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        labels = labels.to(device)

        out = model(input_ids=input_ids,
                    attention_mask=attention_mask,
                    token_type_ids=token_type_ids)

        loss = criterion(out, labels)  # 损失值
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if i % 30 == 0:
            out = out.argmax(dim=1)
            accuracy = (out == labels).sum().item() / len(labels)
            print(i, loss.item(), accuracy)

    ###########################################################################
    # step 2:
    if epoch >= swa_start:
        swa_model.update_parameters(model)
        swa_scheduler.step()
    ###########################################################################
    else:
        scheduler.step()  # 可选

###########################################################################
# step 3:
# Update bn statistics for the swa_model at the end
torch.optim.swa_utils.update_bn(loader, swa_model)
###########################################################################

**************************epoch:0**************************
0 0.7231751084327698 0.453125
30 0.6174258589744568 0.796875
60 0.6006608605384827 0.765625
90 0.5444361567497253 0.828125
120 0.5156898498535156 0.828125
**************************epoch:1**************************
0 0.5752296447753906 0.734375
30 0.5495079755783081 0.734375
60 0.5202062129974365 0.828125
90 0.5549397468566895 0.734375
120 0.46968603134155273 0.859375
**************************epoch:2**************************
0 0.46963077783584595 0.828125
30 0.45827022194862366 0.859375
60 0.5358661413192749 0.796875
90 0.5196751952171326 0.796875
120 0.47215932607650757 0.84375
**************************epoch:3**************************
0 0.5186491012573242 0.796875
30 0.4394235908985138 0.90625
60 0.5121716856956482 0.796875
90 0.5226370692253113 0.765625
120 0.493891179561615 0.828125
**************************epoch:4**************************
0 0.49960458278656006 0.828125
30 0.4899356961250305 0.8125
60 0.49246999621391

In [9]:
# 模型验证

loader_test = torch.utils.data.DataLoader(dataset=Dataset_obj('validation'),
                                          batch_size=64,
                                          collate_fn=collate_fn)


def test(model_obj, loader_dataset):
    model.eval()
    correct = 0
    total = 0
    for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader_dataset):
        # 数据设备切换
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        labels = labels.to(device)

        if i == 5:
            break
        with torch.no_grad():
            out = model_obj(input_ids=input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids)

        out = out.argmax(dim=1)
        correct += (out == labels).sum().item()
        total += len(labels)

    return correct / total


print(test(model, loader_test))

###########################################################################
# step 4:
# Use swa_model to make predictions on test data
print(test(swa_model, loader_test))  # 测试数据集效果得到提升(泛化效果更好)
###########################################################################

0.840625
0.84375
